In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.constraints import MaxNorm

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, matthews_corrcoef, recall_score, precision_score, f1_score, accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt


2024-01-06 00:37:47.716075: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 00:37:47.755277: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-06 00:37:47.755313: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-06 00:37:47.756466: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-06 00:37:47.763755: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 00:37:47.765867: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
BASE_PATH = 'data/ROSMAP/'
methy_path = os.path.join(BASE_PATH,'methy.csv')
mirna_path = os.path.join(BASE_PATH,'mirna.csv')
mrna_path = os.path.join(BASE_PATH,'mrna.csv')
if not os.path.exists(methy_path) or not os.path.exists(mirna_path) or not os.path.exists(mrna_path):
    raise Exception('File not exists!')

In [3]:
methy_df = pd.read_csv(methy_path, index_col=0)
mirna_df = pd.read_csv(mirna_path, index_col=0)
mrna_df = pd.read_csv(mrna_path, index_col=0)
print(methy_df.shape, mirna_df.shape, mrna_df.shape)

(351, 202) (351, 202) (351, 202)


In [4]:
mirna_df.head()

,cg00095526,cg00174500,cg00398048,cg00597076,cg00727590,cg00745735,cg00754253,cg00777121,cg00830029,cg00840516,...,cg25947945,cg26143719,cg26984805,cg27016307,cg27091787,cg27120999,cg27146152,cg27243140,Label,Split
1,0.598012,0.529952,0.232042,0.704232,0.372624,0.875641,0.613827,0.398456,0.440477,0.569050,...,0.609315,0.531111,0.654504,0.472625,0.537309,0.277879,0.396369,0.322722,0.0,1.0
2,0.586543,0.579997,0.299445,0.664964,0.362864,0.890115,0.512908,0.340878,0.444445,0.525837,...,0.596808,0.511755,0.647161,0.484705,0.544367,0.363328,0.385566,0.335697,1.0,1.0
3,0.634034,0.477586,0.274113,0.713400,0.313916,0.886373,0.659061,0.364959,0.416631,0.504891,...,0.558276,0.519758,0.685954,0.467882,0.534083,0.236898,0.447631,0.311407,1.0,1.0
4,0.557265,0.549105,0.352897,0.685386,0.385889,0.976137,0.573691,0.376060,0.445305,0.534050,...,0.644223,0.477909,0.662154,0.529904,0.526594,0.341244,0.427688,0.350926,0.0,1.0
5,0.587819,0.517650,0.309449,0.692494,0.348508,0.901882,0.620659,0.326728,0.454191,0.515826,...,0.572207,0.521734,0.655149,0.519307,0.494170,0.275346,0.408960,0.310059,0.0,1.0


In [5]:
methy_df_wol = methy_df.drop('Label', axis=1)
mirna_df_wol = mirna_df.drop('Label', axis=1)
mrna_df_wol = mrna_df.drop('Label', axis=1)
methy_df_wos = methy_df_wol.drop('Split', axis=1)
mirna_df_wos = mirna_df_wol.drop('Split', axis=1)
mrna_df_wos = mrna_df_wol.drop('Split', axis=1)

combined_df = pd.concat([methy_df_wos, mirna_df_wos, mrna_df_wos], axis=1)
combined_df['Label'] = methy_df['Label']
combined_df['Split'] = methy_df['Split']
print(mrna_df_wol.shape, mrna_df_wos.shape)

(351, 201) (351, 200)


In [6]:
def get_train_test(df):
    train_df = df[df['Split'] == 1].drop('Split', axis=1)
    test_df = df[df['Split'] == 0].drop('Split', axis=1)
    y_train = train_df.pop('Label')
    y_test = test_df.pop('Label')
    return train_df.values, test_df.values, y_train.values, y_test.values

mirna_train, mirna_test, mirna_y,mirna_y_test = get_train_test(mirna_df)
methy_train, methy_test, methy_y,methy_y_test = get_train_test(methy_df)
mrna_train, mrna_test, mrna_y,mrna_y_test = get_train_test(mrna_df)
print(mirna_train.shape, methy_train.shape, mrna_train.shape)
print(mirna_test.shape, methy_test.shape, mrna_test.shape)
print(mirna_y.shape, methy_y.shape, mrna_y.shape)
print(mirna_y_test.shape, methy_y_test.shape, mrna_y_test.shape)

(245, 200) (245, 200) (245, 200)
(106, 200) (106, 200) (106, 200)
(245,) (245,) (245,)
(106,) (106,) (106,)


In [7]:
def scale_dataset(train, test):
    scaler = StandardScaler()
    scaler.fit(train)
    train = scaler.transform(train)
    test = scaler.transform(test)
    return train, test

In [8]:
mirna_train, mirna_test = scale_dataset(mirna_train, mirna_test)
methy_train, methy_test = scale_dataset(methy_train, methy_test)
mrna_train, mrna_test = scale_dataset(mrna_train, mrna_test)
print(mirna_train.shape, methy_train.shape, mrna_train.shape)
print(mirna_test.shape, methy_test.shape, mrna_test.shape)
print(mirna_y.shape, methy_y.shape, mrna_y.shape)
print(mirna_y_test.shape, methy_y_test.shape, mrna_y_test.shape)

(245, 200) (245, 200) (245, 200)
(106, 200) (106, 200) (106, 200)
(245,) (245,) (245,)
(106,) (106,) (106,)


In [28]:
class FeatureExtractor(tf.keras.models.Model):
    def __init__(self, input_shape):
        super().__init__()
        self.input_layer = Input(shape=input_shape)
        self.dense_1 = Dense(32, activation='relu')
        self.dense_2 = Dense(16, activation='relu')
    def call(self, x):
        x = self.dense_1(x)
        x = self.dense_2(x)
        return x

class CombinationNetwork(tf.keras.models.Model):

    @staticmethod
    def get_dense_block(output_dim, activation = 'relu', dropout = 0.5):
        return Sequential([
            Dense(output_dim, activation = activation),
            Dropout(dropout),
        ])
    def __init__(self, input_1_shape, input_2_shape, input_3_shape):
        super().__init__()
        self.ft1 = FeatureExtractor(input_1_shape)
        self.ft2 = FeatureExtractor(input_2_shape)
        self.ft3 = FeatureExtractor(input_3_shape)
        self.fc1 = CombinationNetwork.get_dense_block(32)
        self.fc2 = CombinationNetwork.get_dense_block(16)
        self.out = Dense(1, activation = 'sigmoid')
    
    def call(self, x_arr):
        x1,x2,x3 = x_arr
        x1 = self.ft1(x1)
        x2 = self.ft2(x2)
        x3 = self.ft3(x3)
        x = tf.concat([x1,x2,x3], axis=1)
        x = self.fc1(x)
        x = self.fc2(x)
        return self.out(x)
input_1 = tf.convert_to_tensor(np.random.random((10,200)))
input_2 = tf.convert_to_tensor(np.random.random((10,200)))
input_3 = tf.convert_to_tensor(np.random.random((10,200)))
model = CombinationNetwork((200,), (200,), (200,))
model([input_1, input_2, input_3]).shape

TensorShape([10, 1])

In [29]:
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [30]:
def generator(mirna_data, methy_data, mrna_data, y, batch_size):
    for i in range(0, len(mirna_data), batch_size):
        mirna_batch = mirna_data[i:i+batch_size]
        methy_batch = methy_data[i:i+batch_size]
        mrna_batch = mrna_data[i:i+batch_size]
        y_batch = y[i:i+batch_size]
        yield [mirna_batch, methy_batch, mrna_batch], y_batch

In [32]:
model.metrics_names

['loss', 'accuracy']

In [31]:
n_epochs = 50
for epoch in range(n_epochs):
    for step, data in enumerate(generator(mirna_train, methy_train, mrna_train, mirna_y, 32)):
        x, y = data
        model.train_on_batch(x, y)
        if step % 100 == 0:
            print('Epoch: {}, Step: {}, Loss: {}'.format(epoch, step, model.evaluate(x, y)))
    for step, val_data in enumerate(generator(mirna_test, methy_test, mrna_test, mirna_y_test, 106)):
        x_val, y_val = val_data
        print('Validation: Epoch: {}, Step: {}, Loss: {}'.format(epoch, step, model.evaluate(x_val, y_val)))

1/1 [==============================] - 0s 136ms/step - loss: 0.6294 - accuracy: 0.6562
Epoch: 0, Step: 0, Loss: [0.6293992400169373, 0.65625]
4/4 [==============================] - 0s 4ms/step - loss: 0.6757 - accuracy: 0.5189
Validation: Epoch: 0, Step: 0, Loss: [0.6757262349128723, 0.5188679099082947]
1/1 [==============================] - 0s 26ms/step - loss: 0.5967 - accuracy: 0.7188
Epoch: 1, Step: 0, Loss: [0.5966951251029968, 0.71875]
4/4 [==============================] - 0s 2ms/step - loss: 0.6564 - accuracy: 0.6038
Validation: Epoch: 1, Step: 0, Loss: [0.6563862562179565, 0.6037735939025879]
1/1 [==============================] - 0s 24ms/step - loss: 0.5716 - accuracy: 0.7188
Epoch: 2, Step: 0, Loss: [0.5715963244438171, 0.71875]
4/4 [==============================] - 0s 2ms/step - loss: 0.6446 - accuracy: 0.6415
Validation: Epoch: 2, Step: 0, Loss: [0.6446277499198914, 0.6415094137191772]
1/1 [==============================] - 0s 23ms/step - loss: 0.5540 - accuracy: 0.7500
E